In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from reportlab.pdfbase.pdfmetrics import stringWidth
from reportlab.lib.pagesizes import letter, landscape
from reportlab.pdfgen import canvas
import re

# Set up LaTeX rendering options for matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'cm'  # Set font for LaTeX rendering
latex_image_dir = "latex_eqn_images"  # Directory to store generated images of LaTeX equations
os.makedirs(latex_image_dir, exist_ok=True)  # Ensure directory exists

# Load data from CSV file and select the last 5 rows
data = pd.read_csv("QUE4.csv")
# df = data.tail(5)

# Function to render LaTeX string as an image file
def render_latex_to_image(latex_string, filename, img_width=50):
    # Create a figure for rendering the LaTeX string
    fig = plt.figure(figsize=(img_width/100, 0.5))
    fig.text(x=0.5, y=0.5, s=latex_string, fontsize=11, ha='center', va='center')
    # Save the figure as an image with high DPI for clarity
    fig.savefig(filename, dpi=500, pad_inches=0.03, bbox_inches='tight')
    plt.close(fig)  # Close the figure to free up resources

# Function to wrap text into lines that fit within a specified width
def wrap_text(text, max_width):
    # Split the text into parts, separating LaTeX equations and regular text
    words = re.split(r'(\$.*?\$)', text)
    lines = []
    current_line = ""

    for word in words:
        # If word is a LaTeX equation, add it as a separate line
        if re.match(r'\$.*?\$', word):
            if current_line:  # Append the current line if it contains text
                lines.append(current_line.strip())
                current_line = ""
            lines.append(word)  # Add the LaTeX equation as its own line
        else:  # If word is regular text, break it down into sub-words
            for sub_word in word.split():
                # Check if adding this word to the current line fits within max width
                if stringWidth(current_line + " " + sub_word, "Helvetica", 10) < max_width:
                    current_line += " " + sub_word
                else:
                    # If it doesn't fit, add the current line and start a new line
                    lines.append(current_line.strip())
                    current_line = sub_word  # Start new line with current word
    if current_line:  # Add any remaining text in current_line to lines
        lines.append(current_line.strip())
    
    return lines

# Function to generate a PDF with LaTeX-rendered images and wrapped text
def create_pdf_with_latex(df, filename="output_fixed_wrap_text_and_LaTeX_Eqn.pdf"):
    c = canvas.Canvas(filename, pagesize=landscape(letter))  # Create PDF canvas
    width, height = landscape(letter)  # Get page dimensions

    # Set layout margins and spacing parameters
    left_margin = 50
    top_margin = height - 50
    line_spacing = 30
    max_line_width = 575
    min_y_position = 50  # Minimum Y position to avoid text running off the page

    # Loop through each row in the dataframe
    for index, row in df.iterrows():
        y_position = top_margin  # Reset Y position for each row

        # Loop through each column and value in the row
        for col, value in row.items():
            text = f"{col}: {value}"  # Create text string for column and value
            lines = wrap_text(text, max_line_width)  # Wrap text to fit within max width
            x_position = left_margin  # Reset X position for each row

            # Loop through each wrapped line
            for line in lines:
                # Ensure y_position stays within the printable area
                if y_position <= min_y_position:
                    c.showPage()  # Start a new page if current Y position is too low
                    y_position = top_margin
                    x_position = left_margin

                if re.match(r'\$.*?\$', line):  # If the line contains a LaTeX equation
                    # Render LaTeX to image if not already done
                    latex_image_path = os.path.join(latex_image_dir, f"latex_{index}.png")
                    if not os.path.exists(latex_image_path):
                        render_latex_to_image(line, latex_image_path, img_width=70)

                    image_width = 70  # Define image width
                    if x_position + image_width <= max_line_width:
                        # Draw the image on the current line if it fits
                        c.drawImage(latex_image_path, x_position + 40, y_position - 10, width=image_width, height=30)
                        x_position += image_width + 45
                    else:
                        # Move to the next line if image doesn't fit
                        y_position -= line_spacing
                        x_position = left_margin
                        c.drawImage(latex_image_path, x_position, y_position - 10, width=image_width, height=30)
                        x_position += image_width + 5

                else:
                    # For regular text, measure the line width and position it accordingly
                    line_width = stringWidth(line, "Helvetica", 10)
                    if x_position + line_width <= max_line_width:
                        # Draw the text on the same line if it fits
                        c.drawString(x_position, y_position, line)
                        x_position += line_width + 5
                    else:
                        # Move to the next line if text doesn't fit
                        y_position -= line_spacing
                        x_position = left_margin
                        c.drawString(x_position, y_position, line)
                        x_position += line_width + 5

            y_position -= line_spacing  # Add extra space after each row
        c.showPage()  # Move to a new page after each row

    c.save()  # Save and close the PDF file

# Run the PDF generation function
create_pdf_with_latex(data)


In [18]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.pdfmetrics import stringWidth
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib.pagesizes import letter, landscape
from reportlab.pdfgen import canvas
import re
from PIL import Image

# Set up LaTeX rendering options for matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'cm'
pdfmetrics.registerFont(TTFont("Lato","fonts/Lato-Regular.ttf"))
latex_image_dir = "latex_eqn_images_1"
os.makedirs(latex_image_dir, exist_ok=True)

# Load data from CSV file and select the last 5 rows
df = pd.read_csv("linears2.csv")
data = df.iloc[:, :-1]

# Function to render LaTeX string as an image file
def render_latex_to_image(latex_string, filename, base_img_width=100):
    img_width = base_img_width + len(latex_string) * 2
    fig = plt.figure(figsize=(img_width/100, 0.5))
    fig.text(x=0.5, y=0.5, s=latex_string, fontsize=14, ha='center', va='center')
    fig.savefig(filename, dpi=500, pad_inches=0.03, bbox_inches='tight')
    plt.close(fig)

# Function to wrap text into lines that fit within a specified width
def wrap_text(text, max_width):
    words = re.split(r'(\$.*?\$)', text)
    lines = []
    current_line = ""

    for word in words:
        if re.match(r'\$.*?\$', word):
            if current_line:
                lines.append(current_line.strip())
                current_line = ""
            lines.append(word)
        else:
            for sub_word in word.split():
                if stringWidth(current_line + " " + sub_word, "Lato", 16) < max_width:
                    current_line += " " + sub_word
                else:
                    lines.append(current_line.strip())
                    current_line = sub_word
    if current_line:
        lines.append(current_line.strip())
    
    return lines

# Function to draw a justified string
def draw_justified_string(c, text, x_position, y_position, max_line_width, font_name="Lato", font_size=16):
    words = text.split()
    space_width = stringWidth(" ", font_name, font_size)
    line_width = sum(stringWidth(word, font_name, font_size) for word in words) + space_width * (len(words) - 1)
    
    if line_width >= max_line_width or len(words) == 1:
        c.drawString(x_position, y_position, text)
        return

    extra_space = (max_line_width - line_width) / (len(words) - 1)
    x = x_position

    for word in words:
        c.drawString(x, y_position, word)
        x += stringWidth(word, font_name, font_size) + space_width + extra_space

# Main function to create PDF with justified text for 'question name' column
def create_pdf_with_latex(df, filename="output_fixed_wrap_text_and_LaTeX_Eqn_increased_font_lin_eqn_test.pdf"):
    c = canvas.Canvas(filename, pagesize=landscape(letter))
    width, height = landscape(letter)
    c.setFont("Lato", 16)

    left_margin = 50
    top_margin = height - 50
    line_spacing = 30
    max_line_width = width - 2 * left_margin
    min_y_position = 50

    for index, row in df.iterrows():
        y_position = top_margin

        for col, value in row.items():
            text = f"{col}: {value}"
            lines = wrap_text(text, max_line_width)
            x_position = left_margin

            for line in lines:
                if y_position <= min_y_position:
                    c.showPage()
                    y_position = top_margin
                    x_position = left_margin

                if col == "question name":
                    draw_justified_string(c, line, x_position, y_position, max_line_width)
                elif re.match(r'\$.*?\$', line):
                    latex_image_path = os.path.join(latex_image_dir, f"latex_{index}.png")
                    if not os.path.exists(latex_image_path):
                        render_latex_to_image(line, latex_image_path)

                    with Image.open(latex_image_path) as img:
                        image_width, image_height = img.size
                        base_scale = 0.05
                        scaling_factor = base_scale + len(line) * 0.005
                        scaled_width = image_width * scaling_factor
                        scaled_height = image_height * scaling_factor

                    if x_position + image_width <= max_line_width:
                        c.drawImage(latex_image_path, x_position , y_position - 10, width=scaled_width, height=scaled_height)
                        x_position += image_width + 45
                    else:
                        y_position -= line_spacing
                        x_position = left_margin
                        c.drawImage(latex_image_path, x_position, y_position - 10, width=scaled_width, height=scaled_height)
                        x_position += image_width + 5
                else:
                    line_width = stringWidth(line, "Lato", 16)
                    if x_position + line_width <= max_line_width:
                        c.drawString(x_position, y_position, line)
                        x_position += line_width + 5
                    else:
                        y_position -= line_spacing
                        x_position = left_margin
                        c.drawString(x_position, y_position, line)
                        x_position += line_width + 5

            y_position -= line_spacing
        c.showPage()
        c.setFont("Lato", 16) #Reset font for new Page 

    c.save()

# Run the function to create PDF
create_pdf_with_latex(data)


In [17]:
df = pd.read_csv("linears.csv")
df

,Question,Option A,Option B,Option C,Option D,Correct Option
0,Solve x: $2x + 3 = \frac{x - 1}{2}\$,$\frac{-7}{3}$,$\frac{9}{2}$,2,6,"""A"""
1,Solve x: $8x - 6 = 3(2x + 1)\$,$\frac{-7}{3}$,$\frac{9}{2}$,2,6,"""B"""
2,Solve x: $\frac{3x + 2}{4} = 2\$,$\frac{-7}{3},"$\frac{9}{2}$""",2,6,"""C"""
3,Solve x: $\frac{5x}{3} = x + 4\$,$\frac{-7}{3}$,$\frac{9}{2}$,2,6,"""D"""
4,Solve x: $7x + 5 = \frac{4x + 1}{2}\$,$\frac{-9}{10}$,-5,"$\frac{-20}{7}$""",$\frac{13}{3}$,"""A"""
5,Solve x: $4(x - 2) = 2(3x + 1)\$,$\frac{-}{10}$,-5,$\frac{-20}{7}$,$\frac{13}{3}$,"""B"""
6,Solve x: $\frac{2x - 5}{3} - \frac{3x + 2}{4} ...,$\frac{-9}{10}$,-5,$\frac{-20}{7}$,$\frac{13}{3}$,"""C"""
7,Solve x: $\frac{x + 2}{5} + \frac{x - 3}{2} = ...,$\frac{-9}{10}$,-5,$\frac{-20}{7}$,$\frac{13}{3}$,"""D"""


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.pdfmetrics import stringWidth
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib.pagesizes import letter, landscape
from reportlab.pdfgen import canvas
import re

# Set up LaTeX rendering options for matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'cm'
pdfmetrics.registerFont(TTFont("Lato","fonts/Lato-Regular.ttf"))
latex_image_dir = "latex_eqn_images2"
os.makedirs(latex_image_dir, exist_ok=True)

# Load data from CSV file and select the last 5 rows
df = pd.read_csv("linears.csv")
data = df.iloc[:,:-1]

# Function to render LaTeX string as an image file
def render_latex_to_image(latex_string, filename, img_width=165):
    fig = plt.figure(figsize=(img_width/100, 0.5))
    fig.text(x=0.5, y=0.5, s=latex_string, fontsize=12, ha='center', va='center')
    fig.savefig(filename, dpi=500, pad_inches=0.03, bbox_inches='tight')
    plt.close(fig)

# Function to wrap in latex syntax
def wrap_as_latex(text):
    return f"${text}$"

# Function to wrap text into lines that fit within a specified width
def wrap_text(text, max_width):
    words = re.split(r'\$(.*?)\$', text)
    lines = []
    current_line = ""

    for word in words:
        if re.match(r'\$(.*?)\$', word):
            if current_line:
                lines.append(current_line.strip())
                current_line = ""
            lines.append(word)
        else:
            for sub_word in word.split():
                if stringWidth(current_line + " " + sub_word, "Lato", 16) < max_width:
                    current_line += " " + sub_word
                else:
                    lines.append(current_line.strip())
                    current_line = sub_word
    if current_line:
        lines.append(current_line.strip())
    
    return lines

# Function to draw a justified string
def draw_justified_string(c, text, x_position, y_position, max_line_width, font_name="Lato", font_size=16):
    words = text.split()
    space_width = stringWidth(" ", font_name, font_size)
    line_width = sum(stringWidth(word, font_name, font_size) for word in words) + space_width * (len(words) - 1)
    
    if line_width >= max_line_width or len(words) == 1:
        c.drawString(x_position, y_position, text)
        return

    extra_space = (max_line_width - line_width) / (len(words) - 1)
    x = x_position

    for word in words:
        c.drawString(x, y_position, word)
        x += stringWidth(word, font_name, font_size) + space_width + extra_space

# Main function to create PDF with justified text for 'question name' column
def create_pdf_with_latex(df, filename="output_fixed_wrap_text_and_LaTeX_Eqn_increased_fontttt.pdf"):
    c = canvas.Canvas(filename, pagesize=landscape(letter))
    width, height = landscape(letter)
    c.setFont("Lato", 16)

    left_margin = 50
    top_margin = height - 50
    line_spacing = 50
    max_line_width = width - 2 * left_margin
    min_y_position = 50

    for index, row in df.iterrows():
        y_position = top_margin

        for col, value in row.items():
            text = f"{col}: {value}"
            lines = wrap_text(text, max_line_width)
            x_position = left_margin

            for line in lines:
                if y_position <= min_y_position:
                    c.showPage()
                    y_position = top_margin
                    x_position = left_margin

                if col == "question name":
                    draw_justified_string(c, line, x_position, y_position, max_line_width)
                elif re.match(r'\$(.*?)\$', line):
                    latex_image_path = os.path.join(latex_image_dir, f"latex_{index}.png")
                    if not os.path.exists(latex_image_path):
                        wrapped_latex = wrap_as_latex(line)
                        render_latex_to_image(wrapped_latex, latex_image_path, img_width=165)

                    image_width = 165
                    if x_position + image_width <= max_line_width:
                        c.drawImage(latex_image_path, x_position + 40, y_position - 10, width=image_width, height=50)
                        x_position += image_width + 45
                    else:
                        y_position -= line_spacing
                        x_position = left_margin
                        c.drawImage(latex_image_path, x_position, y_position - 10, width=image_width, height=50)
                        x_position += image_width + 5
                else:
                    line_width = stringWidth(line, "Lato", 16)
                    if x_position + line_width <= max_line_width:
                        c.drawString(x_position, y_position, line)
                        x_position += line_width + 5
                    else:
                        y_position -= line_spacing
                        x_position = left_margin
                        c.drawString(x_position, y_position, line)
                        x_position += line_width + 5

            y_position -= line_spacing
        c.showPage()
        c.setFont("Lato", 16) #Reset font for new Page 

    c.save()

# Run the function to create PDF
create_pdf_with_latex(data)


In [12]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.pdfmetrics import stringWidth
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib.pagesizes import letter, landscape
from reportlab.pdfgen import canvas
from PIL import Image
import re

# Set up LaTeX rendering options for matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'cm'
pdfmetrics.registerFont(TTFont("Lato","fonts/Lato-Regular.ttf"))
latex_image_dir = "latex_eqn_images2"
os.makedirs(latex_image_dir, exist_ok=True)

# Load data from CSV file and select the last 5 rows
df = pd.read_csv("linears.csv")
data = df.iloc[:, :-1]

# Function to render LaTeX string as an image file
def render_latex_to_image(latex_string, filename, img_width=200):
    fig = plt.figure(figsize=(img_width / 100, 0.5))
    fig.text(x=0.5, y=0.5, s=latex_string, fontsize=12, ha='center', va='center')
    fig.savefig(filename, dpi=500, pad_inches=0.03, bbox_inches='tight')
    plt.close(fig)

# Function to wrap text into lines that fit within a specified width
def wrap_text(text, max_width):
    words = re.split(r'(\$.*?\$)', text)  # Split by LaTeX segments within $...$
    lines = []
    current_line = ""

    for word in words:
        if re.match(r'\$.*?\$', word):  # Detect LaTeX segments
            if current_line:
                lines.append(current_line.strip())
                current_line = ""
            lines.append(word)  # Add LaTeX segment as a separate line
        else:
            for sub_word in word.split():
                if stringWidth(current_line + " " + sub_word, "Lato", 16) < max_width:
                    current_line += " " + sub_word
                else:
                    lines.append(current_line.strip())
                    current_line = sub_word
    if current_line:
        lines.append(current_line.strip())
    
    return lines

# Function to draw a justified string
def draw_justified_string(c, text, x_position, y_position, max_line_width, font_name="Lato", font_size=16):
    words = text.split()
    space_width = stringWidth(" ", font_name, font_size)
    line_width = sum(stringWidth(word, font_name, font_size) for word in words) + space_width * (len(words) - 1)
    
    if line_width >= max_line_width or len(words) == 1:
        c.drawString(x_position, y_position, text)
        return

    extra_space = (max_line_width - line_width) / (len(words) - 1)
    x = x_position

    for word in words:
        c.drawString(x, y_position, word)
        x += stringWidth(word, font_name, font_size) + space_width + extra_space

# Main function to create PDF with justified text and LaTeX equations
def create_pdf_with_latex(df, filename="output_dynamic_latex_equations.pdf"):
    c = canvas.Canvas(filename, pagesize=landscape(letter))
    width, height = landscape(letter)
    c.setFont("Lato", 16)

    left_margin = 50
    top_margin = height - 50
    line_spacing = 30
    max_line_width = width - 2 * left_margin
    min_y_position = 50

    for index, row in df.iterrows():
        y_position = top_margin

        for col, value in row.items():
            text = f"{col}: {value}"
            lines = wrap_text(text, max_line_width)
            x_position = left_margin

            for line in lines:
                if y_position <= min_y_position:
                    c.showPage()
                    y_position = top_margin
                    x_position = left_margin

                if re.match(r'\$.*?\$', line):  # Process LaTeX segments
                    latex_image_path = os.path.join(latex_image_dir, f"latex_{index}.png")
                    if not os.path.exists(latex_image_path):
                        render_latex_to_image(line, latex_image_path, img_width=200)

                    image_width = 100  # Set a base width
                    with Image.open(latex_image_path) as img:
                        orig_width, orig_height = img.size
                        scale_factor = 0.5 + len(line)* 0.00001
                        scaled_width = orig_width * scale_factor
                        scaled_height = orig_height * scale_factor
                    
                    # Draw image with dynamic scaling
                    if x_position + scaled_width <= max_line_width:
                        c.drawImage(latex_image_path, x_position, y_position - 10, width=scaled_width, height=scaled_height)
                        x_position += scaled_width + 10
                    else:
                        y_position -= line_spacing
                        x_position = left_margin
                        c.drawImage(latex_image_path, x_position, y_position - 10, width=scaled_width, height=scaled_height)
                        x_position += scaled_width + 10
                else:
                    line_width = stringWidth(line, "Lato", 16)
                    if x_position + line_width <= max_line_width:
                        c.drawString(x_position, y_position, line)
                        x_position += line_width + 5
                    else:
                        y_position -= line_spacing
                        x_position = left_margin
                        c.drawString(x_position, y_position, line)
                        x_position += line_width + 5

            y_position -= line_spacing
        c.showPage()
        c.setFont("Lato", 16)  # Reset font for new page

    c.save()

# Run the function to create PDF
create_pdf_with_latex(data)


KeyboardInterrupt: 